# Initialization vs Isometry in Qiskit

According to `201 Tips and tricks.ipynb` there are exist two approaches to prepare the state: one was proposed by Shende, Bullock and Markov in ["Synthesis of Quantum Logic Circuits"](https://arxiv.org/abs/quant-ph/0406176) (2004) and another — by Iten et al. in ["Quantum Circuits for Isometries"](https://arxiv.org/abs/1501.06911) (2020). For more details please refer to correspondent notebook and original papers.

This notebook concerns comparison of these two methods in terms of how close obtained states are to the desired vector. For someone who knows quantum programming inside out, the answer might appear trivial, but not for me...

### Prerequisites

In [1]:
import math
import numpy as np

from qiskit import QuantumCircuit, BasicAer, execute, transpile
from scipy.stats import wasserstein_distance as ws
from numpy import dot
from numpy.linalg import norm

import warnings
warnings.filterwarnings("ignore")

In [2]:
class Metrics:
    
    def __init__(self):
        self.metrics_available = ["mse", "weierstrass", "cosine_similarity"]
    
    def compute(self, data, dist):
        metrics = dict(
            mse = np.square(data - dist).mean(),
            weierstrass = ws(data, dist),
            cosine_similarity = np.dot(data, dist) / (np.linalg.norm(data) * np.linalg.norm(dist))
        )
        return metrics

In [3]:
class Experiment:
    
    def __init__(self, n_qubits=4, shots=65535):
        self.vector_size = 2 ** n_qubits
        self.n = n_qubits
        self.shots = shots
        self.metrics = Metrics()
        
    def run(self):
        data = self.__get_random_vector()
        
        init_qc = self.__prepare_initialize_circuit(data)
        isom_qc = self.__prepare_isometry_circuit(data)

        result = dict(
            initialize = self.__apply(init_qc, data, self.shots),
            isometry = self.__apply(isom_qc, data, self.shots)
        )
        return result
    
    def __prepare_initialize_circuit(self, data):
        n = self.n
        range_n = range(n)
        qc = QuantumCircuit(n, n)
        qc.initialize(data)
        qc.measure(range_n, range_n)
        return qc
    
    def __prepare_isometry_circuit(self, data):
        n = self.n
        range_n = range(n)
        qc = QuantumCircuit(n, n)
        qc.isometry(data, list(range_n), None)
        qc.measure(range_n, range_n)
        return qc
    
    def __apply(self, qc, data, shots):
        counts = execute(qc, BasicAer.get_backend('qasm_simulator'), shots=shots).result().get_counts()
        dist = self.__get_dist(counts)
        metrics = self.metrics.compute(data, dist)
        return metrics
    
    def __get_dist(self, counts):
        dist = [0. for i in range(self.vector_size)]
        for a, b in sorted(list(counts.items())):
            dist[int(a, 2)] = b
        dist = np.array(dist)
        dist = dist / np.linalg.norm(dist)
        return dist
        
    def __get_random_vector(self):
        a = np.random.rand(self.vector_size)
        a = a / np.linalg.norm(a)
        return a

In [4]:
experiment = Experiment(
    n_qubits=4, 
    shots=10000
)

In [5]:
experiment.run()

{'initialize': {'mse': 0.0061798472912882475,
  'weierstrass': 0.07025012693434898,
  'cosine_similarity': 0.950561221669694},
 'isometry': {'mse': 0.005925654619049145,
  'weierstrass': 0.06916523576196357,
  'cosine_similarity': 0.9525947630476069}}

In [6]:
class Inspector:
    
    def __init__(self, n_qubits=4, shots=65535, iterations=100):
        self.n_qubits = n_qubits
        self.shots = shots
        self.iterations = iterations
        self.metrics_available = Metrics().metrics_available
    
    def run(self):
        metrics_dict_init = {m: list() for m in self.metrics_available}
        metrics_dict_isom = {m: list() for m in self.metrics_available}
        for i in range(self.iterations):
            e = Experiment(
                n_qubits=self.n_qubits,
                shots=self.shots
            )
            e = e.run()
            
            for m, v in e["initialize"].items():
                metrics_dict_init[m].append(v)
                
            for m, v in e["isometry"].items():
                metrics_dict_isom[m].append(v)
        
        print("Statistics for 'qc.initialise':")
        for m, v in metrics_dict_init.items():
            v = np.array(v)
            print(f"\t{m}: {v.mean():.6f} ± {v.var():.6f}")
        
        print("\nStatistics for 'qc.isometry':")
        for m, v in metrics_dict_isom.items():
            v = np.array(v)
            print(f"\t{m}: {v.mean():.6f} ± {v.var():.6f}")
         
        

In [7]:
inspector = Inspector(
    n_qubits=5, 
    shots=65535, 
    iterations=1000
)
inspector.run()

Statistics for 'qc.initialise':
	mse: 0.001959 ± 0.000000
	weierstrass: 0.039325 ± 0.000022
	cosine_similarity: 0.968654 ± 0.000045

Statistics for 'qc.isometry':
	mse: 0.001960 ± 0.000000
	weierstrass: 0.039337 ± 0.000022
	cosine_similarity: 0.968641 ± 0.000045
